In [2]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import dash_table as dt
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from main import AnimalShelter


###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "Password1"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)


df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

### HTML formate for the web-application
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Devyn Mustard'))),
    html.Hr(),
    html.Center(html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image')),
    # Radio options for filters
    html.Div(      
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water-Based Rescue', 'value': 'waterType'},
                {'label': 'Mountain Rescue', 'value': 'mountainType'},
                {'label': 'Disaster Rescue and Tracking', 'value': 'disasterType'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            # Default value on startup
            value='mountainType'
        )
        
    ),
### Data Table
    html.Hr(),
        dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        row_selectable="single",
        selected_rows=[],
        filter_action="native",
        sort_action="native", 
        page_action="native", 
        page_current=0,
        page_size=10,

    ),
    html.Br(),
    html.Hr(),
    
### Geolocation and graph
    html.Div(className='row',
         style={'display' : 'flex', 'background-color': '#333', 'padding': '10px'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={'width': '50%', 'padding-left': '5%', 'padding-right': '5%'}
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={'width': '50%', 'padding-left': '1%'}
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
# Radio button filters
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
        
        #If function if water is selected
        if filter_type == 'waterType':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
        # If function if mountain is selected
        elif filter_type == 'mountainType':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        # If function if disaster is selected
        elif filter_type == 'disasterType':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        # Reset to no filter
        elif filter_type == 'reset':
            df = pd.DataFrame.from_records(shelter.read({}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
             
        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    dff = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(dff, names='breed',)
        )    
    ]

# Call back function to create map
@app.callback(
    Output('map-id', "children"),    
    [Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(rowSelected):
    
    #create map
    if not rowSelected:
        markerArray = (30.75, -97.48) 
        toolTip = "Austin Center"
        popUpHeading = "Austin Center"
        popUpParagraph = "Home"
        
    else: #build the contextual views based on the selection
        dff = pd.DataFrame(df.iloc[rowSelected])
        # create array of coords to input to map
        coordLat = float(dff['location_lat'].to_string().split()[1])
        coordLong = float(dff['location_long'].to_string().split()[1])
        markerArray = (coordLat, coordLong)
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name']

        # Return map icon
    return [dl.Map(style={'width': '700px', 'height': '450px'}, center=markerArray,
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
                                          ])
                                      ])
                                     ])
           ]


app.run_server(debug=True)


Dash app running on http://127.0.0.1:18315/
